# Notebook 4: **DATA CLEANING**

Si importa il dataset contenente i dati acquisiti tramite API. <br>
Nel Notebook 2 i dati sono stati immagazzinati su IMDB in 2 database documentali differenti: *db* e *db2*. <br>
Il database sul quale verrà eseguita la fase di Data Cleaning è chiamato *db_finale* e comprende i dati provenienti da entrambi i database citati. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/Progetto Data Management/MongoDB/mongodb_db/DBFINALE.json"

In [ ]:
import json

In [ ]:
with open(path) as f: 
    db_finale = json.load(f)

In [ ]:
db_finale[0]  # viene mostrato il primo documento del dataset

{'_id': {'$oid': '62b187b5a796fc54c9e4a926'},
 'actorList': [{'asCharacter': 'OFFICER COX',
   'id': 'nm3640496',
   'image': 'https://imdb-api.com/images/original/MV5BMzA4NjE1OTAtOWJiMS00NDM5LWFkMDYtZWI5NmQ1Mjc2ZGQyXkEyXkFqcGdeQXVyMjQwMDg0Ng@@._V1_Ratio1.0000_AL_.jpg',
   'name': 'Wardell Richardson'},
  {'asCharacter': 'On set Voice overs',
   'id': 'nm3719380',
   'image': 'https://imdb-api.com/images/original/MV5BOTJjZDRmYWMtYjQ4YS00MzdkLTk4NDctNzZkM2VmNjc1MThkXkEyXkFqcGdeQXVyMjQwMDg0Ng@@._V1_Ratio1.0000_AL_.jpg',
   'name': 'Christine Hameed'},
  {'asCharacter': 'OFFICER JACOBS',
   'id': 'nm3365443',
   'image': 'https://imdb-api.com/images/original/MV5BYjc1ZmZmMzktM2JjMS00YmE5LTgyNDgtZDYzZWM0MzIwYWFjXkEyXkFqcGdeQXVyMjE0MDk1MzQ@._V1_Ratio1.0000_AL_.jpg',
   'name': 'Don Cano'},
  {'asCharacter': 'Dina',
   'id': 'nm6196306',
   'image': 'https://imdb-api.com/images/original/nopicture.jpg',
   'name': 'Donna Hutchinson'},
  {'asCharacter': 'Antagonist Role',
   'id': 'nm4589197',


In [ ]:
len(db_finale)

13762

In [ ]:
import re

In [ ]:
for documento in db_finale:   # itero per ogni documento della collezione
    
    if documento['awards'] == '':    # awards: splittiamo la variabile se compare '|'. Se è una stringa vuota eliminiamo la chiave
      del documento['awards']
    else:
      documento['awards'] = documento['awards'].split('|')
    
    lista = ['genres', 'languages', 'writers', 'directors', 'companies', 'countries']    # splittiamo la variabili della lista se compare ','. Se è una stringa vuota eliminiamo la chiave
    for chiave in lista:            
      if documento[chiave] == '':
        del documento[chiave]
      else:
        documento[chiave] = documento[chiave].split(',')

    lista1 = ['metacriticRating', 'year']     # trasformiamo in intero i valori delle chiavi in lista1. Se il valore è nullo eliminiamo la chiave   
    for chiave in lista1:          
      if documento[chiave] is None:
        del documento[chiave]
      else:
        documento[chiave] = int(documento[chiave])
    
    if documento['runtimeMins'] is None:    # elimino la chiave runtimeMins se presenta valori nulli, altrimenti trasformo la stringa in un intero. Se la stringa comprende caratteri alfanumerici elimino la chiave.
      del documento['runtimeMins']
    else:
      occ = re.search('\D', documento['runtimeMins'])
      if not occ:
        documento['runtimeMins'] = int(documento['runtimeMins'])
      else:
        del documento['runtimeMins'] 
     
    if documento["imDbRating"] is None:   # imdbRating: trasformiamo in float il valore di imDbRating. Se il valore è nullo eliminiamo la chiave
      del documento["imDbRating"]
    else:
      documento["imDbRating"] = float(documento['imDbRating'])

    
    lista2 = ['budget', 'cumulativeWorldwideGross', 'grossUSA', 'openingWeekendUSA']  # rinomino le chiavi di box-office
    diz = documento['boxOffice']
    for chiave in lista2:
      diz[chiave+"($)"] = diz.pop(chiave)
        
    lista3 = ['cumulativeWorldwideGross($)', 'grossUSA($)', 'openingWeekendUSA($)']   # trasformo in int i valori delle chiavi di box-office, escludendo alcuni caratteri alfanumerici. Se le chiavi presentano delle stringhe vuote elimino le chiavi
    for chiave in lista3:                                                             
      if diz[chiave] == '':
        del diz[chiave]
      else:
        diz[chiave] = int(diz[chiave].replace("$", "").replace(",", ""))
            
    if diz['budget($)'] == '':  
      del diz['budget($)']
    else:
      diz["budget($)"] = int(re.sub('\D', '', diz["budget($)"]))
        
    if documento['boxOffice'] == {}:  # elimino box-office se è un dizionario vuoto
      del documento['boxOffice']

    lista4 = ['_id','companyList', 'countryList', 'directorList', 'errorMessage','fullTitle', 'genreList', 'id', 'imDbRatingVotes', 'image',
              'images', 'keywords', 'languageList', 'originalTitle', 'plotLocal', 'plotLocalIsRtl', 'posters', 'runtimeStr', 'starList',
              'stars', 'tagline', 'trailer', 'tvEpisodeInfo', 'tvSeriesInfo', 'type', 'wikipedia', 'writerList']
    
    for chiave in lista4:   # elimino tutte le chiavi presenti in lista4
      documento.pop(chiave)
                
    if documento['actorList'] == []:   # actorList: elimino la chiave se è una lista vuota, altrimenti, per ogni dizionario della lista, elimino le chiavi 'image' e 'id' e anche 'asCharacter' e 'name' ma queste ultime solo se sono stringhe vuote
      del documento['actorList']       
    else:
      for p in range(len(documento['actorList'])):
        del documento['actorList'][p]['image']
        del documento['actorList'][p]['id']
        if documento['actorList'][p]['asCharacter'] == '':
          del documento['actorList'][p]['asCharacter']
        if documento['actorList'][p]['name'] == '':
          del documento['actorList'][p]['name']

    if documento['similars'] == []:   # similars: elimino la chiave se è una lista vuota, altrimenti, per ogni dizionario della lista, elimino le chiavi 'image' e 'id' e anche 'imDbRating' se è una stringa vuota
      del documento['similars']
    else:
      for p in range(len(documento['similars'])):
        del documento['similars'][p]['image']
        del documento['similars'][p]['id']
        if documento['similars'][p]['imDbRating'] == '':
          del documento['similars'][p]['imDbRating']
        else:
          documento['similars'][p]['imDbRating'] = float(documento['similars'][p]['imDbRating'])

    if 'f rated' in documento['keywordList']:   # keywordList: elimino la keyword 'f rated' dalla lista di keywords, poichè si tratta di un errore. Se la lista è vuota elimino la chiave.
      documento['keywordList'].remove('f rated')
    if documento['keywordList'] == []:
      del documento['keywordList']

    lista6 = ['fullCast', 'ratings', 'releaseDate']   #elimino le chiavi della lista 6 se presentano valori nulli
    for chiave in lista6:
      if documento[chiave] is None:
        del documento[chiave]
    
    if documento['contentRating'] == 'Not Rated' or documento['contentRating'] == 'Unrated' or documento['contentRating'] is None:   # elimino contentRating se presenta valori nulli o non è stato valutato
      del documento['contentRating']                                                                                                 

    lista7 = ['title', 'plot']    # elimino le chiavi della lista7 se i valori sono stringhe vuote
    for chiave in lista7:
      if documento[chiave] == '':
        del documento[chiave]


In [ ]:
# salvo il dataset sistemato in un nuovo file
with open("/content/drive/MyDrive/Progetto Data Management/MongoDB/mongodb_db/DBFINALE_SISTEMATO.json", "w") as final:
    json.dump(db_finale, final)